### Importing Libraries

In [2]:
import pandas as pd
import seaborn as sns
import sqlite3
import os
from IPython.display import HTML
import numpy as np

### Retrieving table from sqlite

In [3]:
conn = sqlite3.connect('nba.sqlite')
cursor = conn.cursor()
conn.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
output_folder = "csv"
os.makedirs(output_folder, exist_ok=True)

In [4]:
# Print the table names
# for table in tables:
#     table_name = table[0]
#     # Read the table data into a pandas DataFrame
#     df = pd.read_sql_query(f"SELECT * FROM {table_name}", conn)
#     output_file = os.path.join(output_folder, f"{table_name}.csv")
#     # Save DataFrame to CSV
#     df.to_csv(output_file, index=False)
# conn.close()

In [5]:
folder_path = "csv/"

# Get a list of all files in the folder
files = os.listdir(folder_path)

# Print the file names
for file in files:
    print(file)


common_player_info.csv
draft_combine_stats.csv
draft_history.csv
game.csv
game_info.csv
game_summary.csv
inactive_players.csv
line_score.csv
officials.csv
other_stats.csv
player.csv
play_by_play.csv
team.csv
team_details.csv
team_history.csv
team_info_common.csv


In [6]:
# "play_by_play.csv"
file_names = ["common_player_info.csv", "draft_combine_stats.csv","draft_history.csv","game.csv","game_info.csv","game_summary.csv","inactive_players.csv","line_score.csv","officials.csv","other_stats.csv","player.csv","team.csv","team_details.csv","team_history.csv","team_info_common.csv"]

### Creating dataframes

In [7]:
dataframes = {}
for file_name in file_names:
    file_path = "csv/" + file_name  # Update with the actual folder path
    df = pd.read_csv(file_path, low_memory=False)
    dataframes[file_name] = df

In [8]:
df_common_player_info = dataframes["common_player_info.csv"]
df_draft_combine_stats = dataframes["draft_combine_stats.csv"]
df_draft_history = dataframes["draft_history.csv"]
df_game= dataframes["game.csv"]
df_game_info = dataframes["game_info.csv"]
df_game_summary = dataframes["game_summary.csv"]
df_inactive_players = dataframes["inactive_players.csv"]
df_line_score = dataframes["line_score.csv"]
df_officials = dataframes["officials.csv"]
df_other_stats = dataframes["other_stats.csv"]
df_player = dataframes["player.csv"]
# df_play_by_play = dataframes["play_by_play.csv"]
df_team = dataframes["team.csv"]
df_team_details = dataframes["team_details.csv"]
df_team_history = dataframes["team_history.csv"]
df_team_info_common = dataframes["team_info_common.csv"]

## Data cleaning: 
df_team, df_team_details, df_team_history, df_team_info_common

In [9]:
#checking data types
#df_team
df_team["year_founded"] = df_team["year_founded"].astype(int)
df_team["state"] = df_team["state"].replace("Atlanta", "Georgia")
df_team.rename(columns={"id":"team_id"}, inplace=True)

# df_team_details
df_team_details[["yearfounded", "arenacapacity"]] = df_team_details[["yearfounded", "arenacapacity"]].astype(pd.Int64Dtype())
new_column_names= {"yearfounded" : "year_founded","arenacapacity": "arena_capacity", "dleagueaffiliation":"league_affiliation"}
df_team_details = df_team_details.rename(columns=new_column_names)
df_team_details.info()
html = (df_team_details.to_html(render_links=True, escape=False))
HTML(html)

# df_team_info_common.info()
# df_team_history


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   team_id             30 non-null     int64 
 1   abbreviation        30 non-null     object
 2   nickname            30 non-null     object
 3   year_founded        30 non-null     Int64 
 4   city                30 non-null     object
 5   arena               30 non-null     object
 6   arena_capacity      20 non-null     Int64 
 7   owner               30 non-null     object
 8   generalmanager      30 non-null     object
 9   headcoach           30 non-null     object
 10  league_affiliation  30 non-null     object
 11  facebook            30 non-null     object
 12  instagram           30 non-null     object
 13  twitter             30 non-null     object
dtypes: Int64(2), int64(1), object(11)
memory usage: 3.5+ KB


,team_id,abbreviation,nickname,year_founded,city,arena,arena_capacity,owner,generalmanager,headcoach,league_affiliation,facebook,instagram,twitter
0,1610612737,ATL,Hawks,1949,Atlanta,State Farm Arena,18729,Tony Ressler,Travis Schlenk,Quin Snyder,College Park Skyhawks,https://www.facebook.com/hawks,https://instagram.com/atlhawks,https://twitter.com/ATLHawks
1,1610612738,BOS,Celtics,1946,Boston,TD Garden,18624,Wyc Grousbeck,Brad Stevens,Joe Mazzulla,Maine Celtics,https://www.facebook.com/bostonceltics,https://instagram.com/celtics,https://twitter.com/celtics
2,1610612739,CLE,Cavaliers,1970,Cleveland,Rocket Mortgage FieldHouse,20562,Dan Gilbert,Koby Altman,JB Bickerstaff,Cleveland Charge,https://www.facebook.com/Cavs,https://instagram.com/cavs,https://twitter.com/cavs
3,1610612740,NOP,Pelicans,2002,New Orleans,Smoothie King Center,,Tom Benson,Trajan Langdon,Willie Green,Birmingham Squadron,https://www.facebook.com/PelicansNBA,https://instagram.com/pelicansnba,https://twitter.com/PelicansNBA
4,1610612741,CHI,Bulls,1966,Chicago,United Center,21711,Jerry Reinsdorf,Arturas Karnisovas,Billy Donovan,Windy City Bulls,https://www.facebook.com/chicagobulls,https://instagram.com/chicagobulls,https://twitter.com/chicagobulls
5,1610612742,DAL,Mavericks,1980,Dallas,American Airlines Center,19200,Mark Cuban,Nico Harrison,Jason Kidd,Texas Legends,https://www.facebook.com/dallasmavs,https://instagram.com/dallasmavs,https://twitter.com/dallasmavs
6,1610612743,DEN,Nuggets,1976,Denver,Ball Arena,,Stan Kroenke,Calvin Booth,Michael Malone,Grand Rapids Gold,https://www.facebook.com/DenverNuggets,https://instagram.com/nuggets,https://twitter.com/nuggets
7,1610612744,GSW,Warriors,1946,Golden State,Chase Center,,Joe Lacob,Bob Myers,Steve Kerr,Santa Cruz Warriors,https://www.facebook.com/warriors,https://instagram.com/warriors,https://twitter.com/warriors
8,1610612745,HOU,Rockets,1967,Houston,Toyota Center,18104,Tilman Fertitta,Rafael Stone,Stephen Silas,Rio Grande Valley Vipers,https://www.facebook.com/houstonrockets,https://instagram.com/houstonrockets,https://twitter.com/HoustonRockets
9,1610612746,LAC,Clippers,1970,Los Angeles,Crypto.com Arena,19060,Steve Ballmer,Michael Winger,Tyronn Lue,Agua Caliente Clippers of Ontario,https://www.facebook.com/LAClippers,https://instagram.com/laclippers,https://twitter.com/LAClippers


In [10]:
# changing the data type
# df_game.info()
df_game["game_date"] = pd.to_datetime(df_game["game_date"])

#df_game_info.info()
df_game_info["game_date"] = pd.to_datetime(df_game_info["game_date"])

#df_game_summary
df_game_summary["game_date_est"] = pd.to_datetime(df_game_summary["game_date_est"])
# df_team_history.info()

In [11]:
#checking the null values
# df_team.isnull().sum()
# df_team_details.isnull().sum()
# df_team_history.isnull().sum()
# df_team_info_common.isnull().sum()

# df_team_history.dropna(axis="index", how="all")

### Joining All The Teams Dataframes

In [12]:
join_team_table_1 = df_team.merge(df_team_details, on =["team_id","abbreviation", "nickname","city", "year_founded"], how = "inner")
join_team_table_2 = join_team_table_1.merge(df_team_info_common, on =["team_id"], how = "inner")
join_team_table_2.drop(columns=["team_city","team_name", "team_abbreviation", "team_code","team_slug"], inplace=True)
join_team_table_2.head(10)

,team_id,full_name,abbreviation,nickname,city,state,year_founded,arena,arena_capacity,owner,...,league_id,season_id,pts_rank,pts_pg,reb_rank,reb_pg,ast_rank,ast_pg,opp_pts_rank,opp_pts_pg
0,1610612737,Atlanta Hawks,ATL,Hawks,Atlanta,Georgia,1949,State Farm Arena,18729,Tony Ressler,...,0,22022,7,117.0,12,43.7,20,24.6,21,116.9
1,1610612738,Boston Celtics,BOS,Celtics,Boston,Massachusetts,1946,TD Garden,18624,Wyc Grousbeck,...,0,22022,4,117.7,6,45.2,7,26.3,9,112.4
2,1610612739,Cleveland Cavaliers,CLE,Cavaliers,Cleveland,Ohio,1970,Rocket Mortgage FieldHouse,20562,Dan Gilbert,...,0,22022,25,112.0,23,41.7,18,24.9,1,106.4
3,1610612740,New Orleans Pelicans,NOP,Pelicans,New Orleans,Louisiana,2002,Smoothie King Center,<NA>,Tom Benson,...,0,22022,16,114.2,14,43.7,11,25.5,16,113.4
4,1610612741,Chicago Bulls,CHI,Bulls,Chicago,Illinois,1966,United Center,21711,Jerry Reinsdorf,...,0,22022,20,113.0,21,42.6,22,24.1,11,112.7
5,1610612742,Dallas Mavericks,DAL,Mavericks,Dallas,Texas,1980,American Airlines Center,19200,Mark Cuban,...,0,22022,17,114.0,30,38.5,29,22.4,14,113.2
6,1610612743,Denver Nuggets,DEN,Nuggets,Denver,Colorado,1976,Ball Arena,<NA>,Stan Kroenke,...,0,22022,5,117.1,16,43.2,2,29.3,10,112.6
7,1610612744,Golden State Warriors,GSW,Warriors,Golden State,California,1946,Chase Center,<NA>,Joe Lacob,...,0,22022,3,118.0,9,44.2,1,29.6,22,117.1
8,1610612745,Houston Rockets,HOU,Rockets,Houston,Texas,1967,Toyota Center,18104,Tilman Fertitta,...,0,22022,29,110.3,4,46.4,28,22.7,27,118.2
9,1610612746,Los Angeles Clippers,LAC,Clippers,Los Angeles,California,1970,Crypto.com Arena,19060,Steve Ballmer,...,0,22022,22,112.7,15,43.4,25,23.2,13,113.0


In [13]:
final_team_table = join_team_table_2.merge(df_team_history, on =["team_id", "nickname","city", "year_founded"], how = "left")
final_team_table.head()

,team_id,full_name,abbreviation,nickname,city,state,year_founded,arena,arena_capacity,owner,...,season_id,pts_rank,pts_pg,reb_rank,reb_pg,ast_rank,ast_pg,opp_pts_rank,opp_pts_pg,year_active_till
0,1610612737,Atlanta Hawks,ATL,Hawks,Atlanta,Georgia,1949,State Farm Arena,18729,Tony Ressler,...,22022,7,117.0,12,43.7,20,24.6,21,116.9,NaN
1,1610612738,Boston Celtics,BOS,Celtics,Boston,Massachusetts,1946,TD Garden,18624,Wyc Grousbeck,...,22022,4,117.7,6,45.2,7,26.3,9,112.4,2019.0
2,1610612739,Cleveland Cavaliers,CLE,Cavaliers,Cleveland,Ohio,1970,Rocket Mortgage FieldHouse,20562,Dan Gilbert,...,22022,25,112.0,23,41.7,18,24.9,1,106.4,2019.0
3,1610612740,New Orleans Pelicans,NOP,Pelicans,New Orleans,Louisiana,2002,Smoothie King Center,<NA>,Tom Benson,...,22022,16,114.2,14,43.7,11,25.5,16,113.4,NaN
4,1610612741,Chicago Bulls,CHI,Bulls,Chicago,Illinois,1966,United Center,21711,Jerry Reinsdorf,...,22022,20,113.0,21,42.6,22,24.1,11,112.7,2019.0


# game, game_info, game_summary

### Data Cleaning(game,game_info, game_summary)

In [14]:
# df_game.info()
# df_game_info.info()
df_game_info["attendance"] = df_game_info["attendance"].astype(pd.Int64Dtype())
# df_game_summary.info()

In [15]:
# df_game.isnull().sum()
# df_game_info.isnull().sum()
# df_game_summary.isnull().sum()

## Joining(game, game_info, game_summary)

In [16]:
join_game_table1 = df_game_info.merge(df_game_summary, on=["game_id"], how="inner")
final_game_table = df_game.merge(join_game_table1, on=["game_id", "game_date"], how="left")

final_game_table


,season_id,team_id_home,team_abbreviation_home,team_name_home,game_id,game_date,matchup_home,wl_home,min,fgm_home,...,game_status_text,gamecode,home_team_id,visitor_team_id,season,live_period,live_pc_time,natl_tv_broadcaster_abbreviation,live_period_time_bcast,wh_status
0,21946,1610610035,HUS,Toronto Huskies,24600001,1946-11-01,HUS vs. NYK,L,0,25.0,...,NaN,19461101/NYKHUS,1.610610e+09,1.610613e+09,1946.0,5.0,NaN,NaN,Q5 -,1.0
1,21946,1610610034,BOM,St. Louis Bombers,24600003,1946-11-02,BOM vs. PIT,W,0,20.0,...,NaN,19461102/PITBOM,1.610610e+09,1.610610e+09,1946.0,4.0,NaN,NaN,Q4 -,1.0
2,21946,1610610032,PRO,Providence Steamrollers,24600002,1946-11-02,PRO vs. BOS,W,0,21.0,...,NaN,19461102/BOSPRO,1.610610e+09,1.610613e+09,1946.0,4.0,NaN,NaN,Q4 -,1.0
3,21946,1610610025,CHS,Chicago Stags,24600004,1946-11-02,CHS vs. NYK,W,0,21.0,...,NaN,19461102/NYKCHS,1.610610e+09,1.610613e+09,1946.0,4.0,NaN,NaN,Q4 -,1.0
4,21946,1610610028,DEF,Detroit Falcons,24600005,1946-11-02,DEF vs. WAS,L,0,10.0,...,NaN,19461102/WASDEF,1.610610e+09,1.610610e+09,1946.0,4.0,NaN,NaN,Q4 -,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62363,22022,1610612755,PHI,Philadelphia 76ers,22201016,2023-03-12,PHI vs. WAS,W,240,41.0,...,Final,20230312/WASPHI,1.610613e+09,1.610613e+09,2022.0,4.0,,NaN,Q4 -,1.0
62364,22022,1610612759,SAS,San Antonio Spurs,22201018,2023-03-12,SAS vs. OKC,L,240,33.0,...,Final,20230312/OKCSAS,1.610613e+09,1.610613e+09,2022.0,4.0,,NaN,Q4 -,1.0
62365,22022,1610612740,NOP,New Orleans Pelicans,22201017,2023-03-12,NOP vs. POR,W,240,43.0,...,Final,20230312/PORNOP,1.610613e+09,1.610613e+09,2022.0,4.0,,NaN,Q4 -,1.0
62366,22022,1610612747,LAL,Los Angeles Lakers,22201019,2023-03-12,LAL vs. NYK,L,240,43.0,...,Final,20230312/NYKLAL,1.610613e+09,1.610613e+09,2022.0,4.0,,ESPN,Q4 - ESPN,1.0


## Data Cleaning (player, common_player_info, inactive_players)

In [17]:
# df_player

df_player= df_player.rename(columns={"id":"player_id"})
# df_player.info()
#df_inactive_players
df_inactive_players["jersey_num"] = df_inactive_players["jersey_num"].replace('-', np.nan).astype(float)
df_inactive_players["jersey_num"] = df_inactive_players["jersey_num"].fillna(-1).astype(int)

# df_inactive_players.info()




# df_common_player_info.info()
df_common_player_info = df_common_player_info.rename(columns={"person_id" : "player_id", "jersey": "jersey_num"})

df_common_player_info["birthdate"] = pd.to_datetime(df_common_player_info["birthdate"])
df_common_player_info["weight"] = df_common_player_info["weight"].replace("nan", 0)
# df_common_player_info[["first_name","jersey_num"]]
# df_common_player_info["position"].unique()
df_common_player_info.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2457 entries, 0 to 2456
Data columns (total 33 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   player_id                         2457 non-null   int64         
 1   first_name                        2457 non-null   object        
 2   last_name                         2457 non-null   object        
 3   display_first_last                2457 non-null   object        
 4   display_last_comma_first          2457 non-null   object        
 5   display_fi_last                   2457 non-null   object        
 6   player_slug                       2457 non-null   object        
 7   birthdate                         2457 non-null   datetime64[ns]
 8   school                            2453 non-null   object        
 9   country                           2457 non-null   object        
 10  last_affiliation                  2457 non-null 

### Joning(player, inactive_players, common_player_info)



In [18]:
# Extract the common columns for merging
common_columns = ["player_id", "first_name", "last_name"]

# Merge df_player and df_common_player_info using pd.concat()
join_player_table1 = pd.concat([df_player[common_columns], df_common_player_info], axis=1)

# Merge df_inactive_players and join_player_table1 using pd.concat()
final_player_table = pd.concat([df_inactive_players[common_columns], join_player_table1], axis=1)

# Print information about the final_player_table DataFrame
final_player_table.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 105217 entries, 0 to 105216
Data columns (total 39 columns):
 #   Column                            Non-Null Count   Dtype         
---  ------                            --------------   -----         
 0   player_id                         105217 non-null  int64         
 1   first_name                        105216 non-null  object        
 2   last_name                         105216 non-null  object        
 3   player_id                         4831 non-null    float64       
 4   first_name                        4825 non-null    object        
 5   last_name                         4831 non-null    object        
 6   player_id                         2457 non-null    float64       
 7   first_name                        2457 non-null    object        
 8   last_name                         2457 non-null    object        
 9   display_first_last                2457 non-null    object        
 10  display_last_comma_first        

### Joining(draft_history, draft_combine_history)

In [19]:
df_draft_history = df_draft_history.rename(columns={"person_id":"player_id"})
# df_draft_combine_stats.info()
join_draft_table1 = df_draft_combine_stats.merge(df_draft_history, on=["player_id","season", "player_name"], how="right")
join_draft_table1.head()


,season,player_id,first_name,last_name,player_name,position,height_wo_shoes,height_wo_shoes_ft_in,height_w_shoes,height_w_shoes_ft_in,...,round_pick,overall_pick,draft_type,team_id,team_city,team_name,team_abbreviation,organization,organization_type,player_profile_flag
0,1947,79299,NaN,NaN,Clifton McNeeley,NaN,NaN,NaN,NaN,NaN,...,1,1,Draft,1610610031,Pittsburgh,Ironmen,PIT,Texas-El Paso,College/University,0
1,1947,78109,NaN,NaN,Glen Selbo,NaN,NaN,NaN,NaN,NaN,...,2,2,Draft,1610610035,Toronto,Huskies,HUS,Wisconsin,College/University,1
2,1947,76649,NaN,NaN,Eddie Ehlers,NaN,NaN,NaN,NaN,NaN,...,3,3,Draft,1610612738,Boston,Celtics,BOS,Purdue,College/University,1
3,1947,79302,NaN,NaN,Walt Dropo,NaN,NaN,NaN,NaN,NaN,...,4,4,Draft,1610610032,Providence,Steamrollers,PRO,Connecticut,College/University,0
4,1947,77048,NaN,NaN,Dick Holub,NaN,NaN,NaN,NaN,NaN,...,5,5,Draft,1610612752,New York,Knicks,NYK,Long Island-Brooklyn,College/University,1


### Other table(play_by_play, other_stats, officials, line_score)

In [20]:
# df_line_score.info()
# df_other_stats

# Exporting the file

In [21]:
# Create the "save" folder if it doesn't exist
if not os.path.exists("export"):
    os.makedirs("export")

final_game_table.to_csv("export/game_table", index=False)
final_team_table.to_csv("export/team_table", index=False)
final_player_table.to_csv("export/player_table", index=False)

## Queries

### Game Table

In [242]:
final_game_table[final_game_table["game_date"] >="2022-01-01"]

,season_id,team_id_home,team_abbreviation_home,team_name_home,game_id,game_date,matchup_home,wl_home,min,fgm_home,...,game_status_text,gamecode,home_team_id,visitor_team_id,season,live_period,live_pc_time,natl_tv_broadcaster_abbreviation,live_period_time_bcast,wh_status
60647,22021,1610612749,MIL,Milwaukee Bucks,22100539,2022-01-01,MIL vs. NOP,W,240,49.0,...,Final,20220101/NOPMIL,1.610613e+09,1.610613e+09,2021.0,4.0,NaN,NBA TV,Q4 - NBA TV,1.0
60648,22021,1610612751,BKN,Brooklyn Nets,22100542,2022-01-01,BKN vs. LAC,L,240,42.0,...,Final,20220101/LACBKN,1.610613e+09,1.610613e+09,2021.0,4.0,NaN,NaN,Q4 -,1.0
60649,22021,1610612765,DET,Detroit Pistons,22100540,2022-01-01,DET vs. SAS,W,265,41.0,...,Final,20220101/SASDET,1.610613e+09,1.610613e+09,2021.0,5.0,NaN,NaN,Q5 -,1.0
60650,22021,1610612745,HOU,Houston Rockets,22100543,2022-01-01,HOU vs. DEN,L,240,37.0,...,Final,20220101/DENHOU,1.610613e+09,1.610613e+09,2021.0,4.0,NaN,NaN,Q4 -,1.0
60651,22021,1610612762,UTA,Utah Jazz,22100544,2022-01-01,UTA vs. GSW,L,240,40.0,...,Final,20220101/GSWUTA,1.610613e+09,1.610613e+09,2021.0,4.0,NaN,NBA TV,Q4 - NBA TV,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62363,22022,1610612755,PHI,Philadelphia 76ers,22201016,2023-03-12,PHI vs. WAS,W,240,41.0,...,Final,20230312/WASPHI,1.610613e+09,1.610613e+09,2022.0,4.0,,NaN,Q4 -,1.0
62364,22022,1610612759,SAS,San Antonio Spurs,22201018,2023-03-12,SAS vs. OKC,L,240,33.0,...,Final,20230312/OKCSAS,1.610613e+09,1.610613e+09,2022.0,4.0,,NaN,Q4 -,1.0
62365,22022,1610612740,NOP,New Orleans Pelicans,22201017,2023-03-12,NOP vs. POR,W,240,43.0,...,Final,20230312/PORNOP,1.610613e+09,1.610613e+09,2022.0,4.0,,NaN,Q4 -,1.0
62366,22022,1610612747,LAL,Los Angeles Lakers,22201019,2023-03-12,LAL vs. NYK,L,240,43.0,...,Final,20230312/NYKLAL,1.610613e+09,1.610613e+09,2022.0,4.0,,ESPN,Q4 - ESPN,1.0


In [233]:
final_game_table[(final_game_table["game_date"] >="2022-05-01") & (final_game_table["game_date"] <="2022-10-31")]

,season_id,team_id_home,team_abbreviation_home,team_name_home,game_id,game_date,matchup_home,wl_home,min,fgm_home,...,game_status_text,gamecode,home_team_id,visitor_team_id,season,live_period,live_pc_time,natl_tv_broadcaster_abbreviation,live_period_time_bcast,wh_status
61350,22022,1610612738,BOS,Boston Celtics,22200001,2022-10-18,BOS vs. PHI,W,240,46.0,...,Final,20221018/PHIBOS,1.610613e+09,1.610613e+09,2022.0,4.0,,TNT,Q4 - TNT,1.0
61351,22022,1610612744,GSW,Golden State Warriors,22200002,2022-10-18,GSW vs. LAL,W,240,45.0,...,Final,20221018/LALGSW,1.610613e+09,1.610613e+09,2022.0,4.0,NaN,TNT,Q4 - TNT,1.0
61352,22022,1610612737,ATL,Atlanta Hawks,22200005,2022-10-19,ATL vs. HOU,W,240,45.0,...,Final,20221019/HOUATL,1.610613e+09,1.610613e+09,2022.0,4.0,,NaN,Q4 -,1.0
61353,22022,1610612754,IND,Indiana Pacers,22200004,2022-10-19,IND vs. WAS,L,240,39.0,...,Final,20221019/WASIND,1.610613e+09,1.610613e+09,2022.0,4.0,,NaN,Q4 -,1.0
61354,22022,1610612762,UTA,Utah Jazz,22200012,2022-10-19,UTA vs. DEN,W,240,42.0,...,Final,20221019/DENUTA,1.610613e+09,1.610613e+09,2022.0,4.0,NaN,NaN,Q4 -,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61447,22022,1610612764,WAS,Washington Wizards,22200097,2022-10-31,WAS vs. PHI,L,240,38.0,...,Final,20221031/PHIWAS,1.610613e+09,1.610613e+09,2022.0,4.0,NaN,NaN,Q4 -,1.0
61448,22022,1610612749,MIL,Milwaukee Bucks,22200100,2022-10-31,MIL vs. DET,W,240,38.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
61449,22022,1610612751,BKN,Brooklyn Nets,22200098,2022-10-31,BKN vs. IND,W,240,43.0,...,Final,20221031/INDBKN,1.610613e+09,1.610613e+09,2022.0,4.0,,NBA TV,Q4 - NBA TV,1.0
61450,22022,1610612762,UTA,Utah Jazz,22200101,2022-10-31,UTA vs. MEM,W,240,42.0,...,Final,20221031/MEMUTA,1.610613e+09,1.610613e+09,2022.0,4.0,,NaN,Q4 -,1.0


In [251]:
final_game_table
final_2022 = final_game_table[(final_game_table["matchup_home"] == "GSW vs. BOS") | (final_game_table["matchup_home"] == "BOS vs. GSW") & (final_game_table["game_date"] >= "2021-01-01")].tail()
final_2022[["team_abbreviation_home","team_name_home","game_id","game_date","matchup_home","wl_home","game_date"]]

,team_abbreviation_home,team_name_home,game_id,game_date,matchup_home,wl_home,game_date
59876,BOS,Boston Celtics,22000858,2021-04-17,BOS vs. GSW,W,2021-04-17
60554,BOS,Boston Celtics,22100435,2021-12-17,BOS vs. GSW,L,2021-12-17
61157,GSW,Golden State Warriors,22101042,2022-03-16,GSW vs. BOS,L,2022-03-16
61741,GSW,Golden State Warriors,22200392,2022-12-10,GSW vs. BOS,W,2022-12-10
62027,BOS,Boston Celtics,22200679,2023-01-19,BOS vs. GSW,W,2023-01-19


In [203]:
team_group = final_game_table.groupby(["team_name_home", "wl_home"])["wl_home"].count().sort_values(ascending=False)
team_group_df = team_group[team_group.index.get_level_values("wl_home") == "W"].to_frame()
team_group_df = team_group_df.rename(columns={"wl_home": "Total"})
team_group_df.head(10)


,,Total
team_name_home,wl_home,
Boston Celtics,W,2085
New York Knicks,W,1748
Los Angeles Lakers,W,1741
Detroit Pistons,W,1501
Philadelphia 76ers,W,1423
Phoenix Suns,W,1411
Chicago Bulls,W,1367
Portland Trail Blazers,W,1366
Milwaukee Bucks,W,1361


In [153]:
indv_team = final_game_table[(final_game_table["team_name_home"] == "Boston Celtics") & (final_game_table["wl_home"] == "W")].count().sort_values(ascending=False)
indv_team

season_id                           2085
ftm_home                            2085
video_available_away                2085
plus_minus_away                     2085
pts_away                            2085
                                    ... 
game_sequence                        642
game_status_text                     628
game_time                            606
natl_tv_broadcaster_abbreviation     133
live_pc_time                          42
Length: 69, dtype: int64

In [132]:
final_2022 = final_game_table[(final_game_table["season"] == 2022.0) & (final_game_table["game_status_text"] == "Final")]
final_date = final_2022["game_date"].idxmax()

final_date = final_2022.loc[final_date, ["team_abbreviation_home","wl_home", "team_abbreviation_away","game_date","matchup_home"]]
print(final_date.to_frame())

                                      62362
team_abbreviation_home                  CHA
wl_home                                   L
team_abbreviation_away                  CLE
game_date               2023-03-12 00:00:00
matchup_home                    CHA vs. CLE


In [71]:
winning_home_sum = final_game_table[["team_name_home", "wl_home","season"]].groupby(["team_name_home","wl_home"]).count()
# .sort_values(by="wl_home",ascending=False)
winning_home_sum.head(20)

season
team_name_home    wl_home        
Anderson Packers  L             8
                  W            25
Atlanta Hawks     L           793
                  W          1289
Baltimore Bullets L           244
                  W           368
Boston Celtics    L           823
                  W          2019
Brooklyn Nets     L           172
                  W           208
Buffalo Braves    L           153
                  W           166
Capital Bullets   L            10
                  W            30
Charlotte Bobcats L           208
                  W           189
Charlotte Hornets L           342
                  W           459
Chicago Bulls     L           823
                  W          1331